In [ ]:
### LLM API Examples
# This script demonstrates how to interact with a LLM API

First of all, ensure all the required libraries are installed

In [ ]:
pip install google-genai streamlit

Setup you API Key

In [ ]:
my_api_key = "YOUR_API_KEY_HERE"

#### Test API Calling

In [ ]:
from google import genai
from google.genai import types

# Initialize the client with your API key
client = genai.Client(api_key= my_api_key)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking, making the response faster but less accurate
    ),
)
print(response.text)

---

#### Ask Agent Loop

In [ ]:
from google import genai
from google.genai import types

# Choose a model. E.g. “gemini-2.5-flash” or whichever you have access to. :contentReference[oaicite:1]{index=1}
MODEL = "gemini-2.5-flash"

def ask_agent(prompt: str) -> str:

    # Initialize the client with your API key
    client = genai.Client(api_key="")

    response = client.models.generate_content(
        model=MODEL,
        contents=prompt
    )
    # The client returns a structure; here we extract the generated text
    return response.text or ""

def run_agent():
    print("Agent ready. Type 'exit' to quit.")
    while True:
        user = input("You: ")
        if user.strip().lower() == "exit":
            break
        reply = ask_agent(user)
        print("Agent:", reply)

if __name__ == "__main__":
    run_agent()

---

#### Simulate ChatGPT

In [ ]:
# app.py
import streamlit as st
import google.generativeai as genai

# -------------------------
# Page Configuration
# -------------------------
st.set_page_config(page_title="Gemini Chatbot with Memory", page_icon="🤖")

# -------------------------
# API Key Input
# -------------------------
api_key = st.text_input("Enter your Gemini API key:", type="password")
if not api_key:
    st.stop()

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

# -------------------------
# Session State (Memory)
# -------------------------
if "memory" not in st.session_state:
    st.session_state.memory = []  # stores (role, text) tuples

# -------------------------
# Sidebar Controls
# -------------------------
st.sidebar.header("⚙️ Settings")
use_memory = st.sidebar.checkbox("Enable Memory", value=True)
if st.sidebar.button("🧹 Clear Chat"):
    st.session_state.memory = []
    st.experimental_rerun()

st.sidebar.markdown(
    """
**Memory ON:** Model sees full chat history.  
**Memory OFF:** Model only sees the latest prompt.
"""
)

# -------------------------
# Chat UI
# -------------------------
st.title("💬 Gemini Chatbot (Memory Toggle Demo)")
st.caption("Demonstrating stateless vs. stateful LLM behavior using Gemini SDK")

# Display existing conversation
for role, text in st.session_state.memory:
    st.chat_message(role).markdown(text)

# User input
if prompt := st.chat_input("Type your message..."):
    st.chat_message("user").markdown(prompt)
    st.session_state.memory.append(("user", prompt))

    # Build input context
    if use_memory:
        # Include full history
        context = "\n".join([f"{r}: {t}" for r, t in st.session_state.memory])
    else:
        # Only include latest user message
        context = f"user: {prompt}"

    # Generate response
    response = model.generate_content(context)
    reply = response.text

    # Append and show
    st.session_state.memory.append(("assistant", reply))
    st.chat_message("assistant").markdown(reply)
